In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

%matplotlib

Using matplotlib backend: TkAgg


In [4]:
import scipy.fftpack
from scipy.signal import gaussian

In [5]:
def f_beta(a,q,beta_guess):
    return q**2/( (beta_guess+2)**2 - a - q**2/( (beta_guess+4)**2 - a - q**2/( (beta_guess+6)**2 - a - q**2/( (beta_guess+8)**2 - a - q**2/( (beta_guess+10)**2 - a - q**2/( (beta_guess+10)**2 - a - q**2/( (beta_guess+12)**2 - a ) ) ) ) ) ) )

def beta_continue_alamano(a,q,beta_guess): #a l'air ok
    return a + f_beta(a,q,beta_guess) + f_beta(a,q,-beta_guess)

# GiantMol

In [6]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide


r0 = 2.865e-3/1.14511 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+
Omega = 2.0e6*2*pi # 2.047e6
bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
# wzLC = (2*pi*100000)**2
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [23]:
q = np.linspace(0.6,0.7,11)
print(q)

[0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7 ]


In [34]:
# Give q
# compute URF, beta, a and UDC for spherical cloud
# compute secular frequencies
K = 1e5*2*pi
# Udc = 10.93 # 2.8628*4   9.014464
q = 0.64 # np.linspace(0.61,0.63,21)[1::2] # np.linspace(0.54,0.70,9)
# np.linspace(0.61,0.63,21) np.linspace(0.61,0.63,21)[1::2] # np.linspace(0.54,0.70,9) # 
# array([0.611, 0.613, 0.615, 0.617, 0.619, 0.621, 0.623, 0.625, 0.627, 0.629])
print('q',q)
Urf = q*(m_Ca*r0**2*Omega**2)/(4*C_e)
print('Urf =',Urf)
beta_guess = 0.423
a = 0
for i in range(50):
    for k in range(50):
        beta_guess = beta_continue_alamano(a,q,beta_guess)
        beta_guess = sqrt(beta_guess)
    omega_r_2 = ( beta_guess*Omega/2 )**2 # m_Ca*d0**2/(3*kappa_simion*C_e)*(0.5*beta_guess*Omega)**2
    a = -2*omega_r_2/Omega**2
# omega_z_2 = omega_r_2
Udc_for_alpha1 = omega_r_2/K**2
omega_z_2 = omega_r_2
Udc_for_alpha1bis = omega_z_2/K**2
print('a =',a)
print('q =',q)
print('beta =',beta_guess)
# Udc_for_alpha1 = omega_r_2/K**2
# # omega_z_2 = K**2 * Udc_for_alpha1
# omega_x_2 = omega_r_2 + 0.5*omega_z_2
print('Udc =',Udc_for_alpha1)
print(Udc_for_alpha1bis)
print(omega_z_2/omega_r_2)

q 0.64
Urf = 65.7033357974304
a = -0.07539548259691725
q = 0.64
beta = 0.38831812369993046
Udc = 15.07909651938345
15.07909651938345
1.0


In [36]:
# Give q
# compute URF, beta, a and UDC for spherical cloud
# compute secular frequencies
K = 1e5*2*pi
# Udc = 10.93 # 2.8628*4   9.014464
Urf = 61.596877310090996 # np.linspace(0.61,0.63,21)[1::2] # np.linspace(0.54,0.70,9)
# np.linspace(0.61,0.63,21) np.linspace(0.61,0.63,21)[1::2] # np.linspace(0.54,0.70,9) # 
# array([0.611, 0.613, 0.615, 0.617, 0.619, 0.621, 0.623, 0.625, 0.627, 0.629])
print('q',q)
q = (4*C_e)/(m_Ca*r0**2*Omega**2) * Urf
print('q =',q)
beta_guess = 0.423
a = 0
for i in range(50):
    for k in range(50):
        beta_guess = beta_continue_alamano(a,q,beta_guess)
        beta_guess = sqrt(beta_guess)
    omega_r_2 = ( beta_guess*Omega/2 )**2 # m_Ca*d0**2/(3*kappa_simion*C_e)*(0.5*beta_guess*Omega)**2
    a = -2*omega_r_2/Omega**2
# omega_z_2 = omega_r_2
Udc_for_alpha1 = omega_r_2/K**2
omega_z_2 = omega_r_2
Udc_for_alpha1bis = omega_z_2/K**2
print('a =',a)
print('q =',q)
print('beta =',beta_guess)
# Udc_for_alpha1 = omega_r_2/K**2
# # omega_z_2 = K**2 * Udc_for_alpha1
# omega_x_2 = omega_r_2 + 0.5*omega_z_2
print('Udc =',Udc_for_alpha1)
print(Udc_for_alpha1bis)
print(omega_z_2/omega_r_2)

q [0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7 ]
q = 0.6000000000000001
a = -0.06541616226969633
q = 0.6000000000000001
beta = 0.36170751241768906
Udc = 13.083232453939267
13.083232453939267
1.0


In [16]:
# Give q and alpha
# compute URF, beta, a and UDC
# compute secular frequencies

K = wzLC
# Udc = 10.93 # 2.8628*4   9.014464
q = 0.64
alpha = 1

print('q',q)
Urf = q*(m_Ca*r0**2*Omega**2)/(4*C_e)
print('Urf =',Urf)
beta_guess = 0.423
a = 0
for i in range(50):
    for k in range(50):
        beta_guess = beta_continue_alamano(a,q,beta_guess)
        beta_guess = sqrt(beta_guess)
    omega_x_2 = (0.5*beta_guess*Omega)**2
    omega_z_2 = alpha/(1+0.5*alpha)*omega_x_2
    omega_r_2 = omega_x_2 - 0.5*omega_z_2
    a = 2*omega_z_2/Omega**2
print('Udc = ',omega_z_2/K)
print('a =',a)
print('q =',q)
print('beta =',beta_guess)
print(sqrt(omega_x_2)/2/pi)
print(sqrt(omega_r_2)/2/pi)
print(sqrt(omega_z_2)/2/pi)
print(omega_z_2/omega_r_2)

q 0.64
Urf = 65.7033357974304
Udc =  41.39846370463104
a = 0.17068402212388706
q = 0.64
beta = 0.7155781343582692
715578.1343582693
584267.1000901678
584267.1000901678
0.9999999999999999


In [56]:
# Give q
# compute URF, beta, a and UDC for spherical cloud
# compute secular frequencies
K = 1e5*2*pi
# Udc = 10.93 # 2.8628*4   9.014464
q = np.linspace(0.54,0.70,9) # 0.6 # 0.64
print('q',q)
Urf = q*(m_Ca*r0**2*Omega**2)/(4*C_e)
print('Urf =',Urf)
beta_guess = 0.423
au = 0
for i in range(15):
    for k in range(50):
        beta_guess = beta_continue_alamano(au,q,beta_guess)
        beta_guess = sqrt(beta_guess)
    omega_r_2 = ( beta_guess*Omega/2 )**2 
    az = 2*omega_r_2/Omega**2
    au = -0.5 * 2*omega_r_2/Omega**2
#     print('aaaauuu',au)
omega_z_2 = -2*au*Omega**2/2
Udc_for_alpha1 = -2*au*Omega**2/2/K**2
print('a =',au)
print('q =',q)
print('beta =',beta_guess)
# Udc_for_alpha1 = omega_r_2/K**2
# # omega_z_2 = K**2 * Udc_for_alpha1
# omega_x_2 = omega_r_2 + 0.5*omega_z_2
print('Udc =',Udc_for_alpha1)
print(omega_z_2/omega_r_2)

q [0.54 0.56 0.58 0.6  0.62 0.64 0.66 0.68 0.7 ]
Urf = [55.43718958 57.49041882 59.54364807 61.59687731 63.65010655 65.7033358
 67.75656504 69.80979428 71.86302353]
a = [-0.03200432 -0.03468859 -0.0375179  -0.04049935 -0.04364082 -0.04695098
 -0.05043951 -0.05411719 -0.05799605]
q = [0.54 0.56 0.58 0.6  0.62 0.64 0.66 0.68 0.7 ]
beta = [0.35779502 0.37249745 0.38739074 0.40248903 0.41780768 0.43336349
 0.44917486 0.46526203 0.48164737]
Udc = [12.80172796 13.87543491 15.00715853 16.19974171 17.45632602 18.78039182
 20.17580548 21.64687542 23.19841905]
[1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [62]:
# Give URF and UDC
K = 1e5*2*pi
print('q',q)
Urf = 61.597 # 62.6235 
Udc = 13.56527568 # 13.56527568  16.1997

q = (4*C_e)/(m_Ca*r0**2*Omega**2)*Urf
print('q =',q)
beta_guess = 0.423
omega_z_2 = K**2*Udc
az = 2*K**2/Omega**2*Udc
au = -0.5 * az
for i in range(15):
    for k in range(50):
        beta_guess = beta_continue_alamano(au,q,beta_guess)
        beta_guess = sqrt(beta_guess)
omega_r_2 = ( beta_guess*Omega/2 )**2
print('au =',au)
print('az =',az)
print('beta =',beta_guess)
print(omega_r_2/omega_z_2)

q 0.6000011950921641
q = 0.6000011950921641
au = -0.0339131892
az = 0.0678263784
beta = 0.41268846328493686
1.255498021168728


In [57]:
# plot with complicated scan plot only crystal part at the end

fig = figure('stab_diag',clear='True')
ax = fig.add_subplot(1, 1, 1)
fig.set_size_inches(11.69,8.27)


ax.plot(q,a0,'-',color='k',label=r'$\Phi_z^{\mathrm{quad}}$ Quadratic pot.',lw=2)
ax.plot(q,b1,'-',color='k',label='$\Phi_z^{\mathrm{GM}}$ GiantMol axial pot.',lw=2)
# ax.plot(q,-a0,'-',color='k',label=r'$\Phi_z^{\mathrm{quad}}$ Quadratic pot.',lw=2)
# ax.plot(q,-b1,'-',color='k',label='$\Phi_z^{\mathrm{GM}}$ GiantMol axial pot.',lw=2)

for k in range(len(nutest)):
    ax.plot(q,a[k],color='k',ls=':')

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.2))
# text(-0.83, 8000, f'FWHM = ${2*sqrt(2*log(2)) * sig[to_plot] * 1000:.2f}$ MHz\n        T = {temperature:.2f} K',
# rotation=0,
# horizontalalignment='left',
# verticalalignment='bottom',
# color='xkcd:vermillion',
# fontsize=17,
#      bbox=dict(facecolor='white'))

ax.set_xlim(-0.02,1.51)
ax.set_ylim(-1.05,1.05)
# ax.set_xlabel(r'$z$ [mm]')
# ax.set_ylabel('$U_{pot}$ [eV]')
# ax.legend(fontsize=16)

ax.grid()
tight_layout()

# savefig('pot_compare_x1',dpi=300)

In [56]:
Urf = 61.596877310090996
Udc =  5.345002954905499
q = Urf/(m_Ca*r0**2*Omega**2)*(4*C_e)
print('q',q)
print('Urf =',Urf)
beta_guess = 0.423
omega_z_2 = wzLC*Udc
a = 2*omega_z_2/Omega**2
for k in range(50):
    beta_guess = beta_continue_alamano(a,q,beta_guess)
    beta_guess = sqrt(beta_guess)
omega_x_2 = (0.5*beta_guess*Omega)**2
omega_r_2 = omega_x_2 - 0.5*omega_z_2
print('Udc = ',omega_z_2/K)
print('a =',a)
print('q =',q)
print('beta =',beta_guess)
print(sqrt(omega_x_2)/2/pi)
print(sqrt(omega_r_2)/2/pi)
print(sqrt(omega_z_2)/2/pi)
print(omega_z_2/omega_r_2)

q 0.6
Urf = 61.596877310090996
Udc =  5.345002954905499
a = 0.022037209136948642
q = 0.6
beta = 0.49235079009425287
492350.79009425285
469438.05914037954
209939.082292691
0.19999999999999998


In [39]:
# spherical cloud
alpha = 108/55

alpha_fact = sqrt(1-alpha**(-2))
fact_alpha = sqrt(alpha**(-2)-1)
if alpha<1:
    omegaz2_sur_omegau2 = -2*( math.asinh(fact_alpha) - alpha*fact_alpha ) / ( math.asinh(fact_alpha) - alpha**(-1)*fact_alpha)
else:
    omegaz2_sur_omegau2 = -2*( math.asin(alpha_fact) - alpha*alpha_fact ) / ( math.asin(alpha_fact) - alpha**(-1)*alpha_fact)
print(omegaz2_sur_omegau2)

2.1847603706437155


/tmp/ipykernel_237210/1034454679.py:5: RuntimeWarning: invalid value encountered in sqrt
  fact_alpha = sqrt(alpha**(-2)-1)


# TADOTI

In [ ]:
r0=3.93e-3 # 2.5e-3
d0=21e-2/2

Omega=5.215e6*2*pi # 2.047e6

mkappa = 9.6*1e-4

In [ ]:
Udc = 1200
Urf = 830/4
omega_z_2 = 2*C_e*mkappa*Udc/m_Ca/d0**2
omega_x_2 = 2*(C_e*Urf/m_Ca/Omega/r0**2)**2
omega_r_2 = omega_x_2 - 0.5*omega_z_2

qx = 4*C_e*Urf/(m_Ca*r0**2*Omega**2)
az = 2*omega_z_2/Omega**2

print('alpha =',omega_z_2/omega_x_2)
print('qx =',qx)
print('ax =',ax)
print('az =',az)
print('f_z =',sqrt(omega_z_2)/2/pi)
print('f_x =',sqrt(omega_x_2)/2/pi)
print('f_r =',sqrt(omega_r_2)/2/pi)